In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
print(tf.__version__)

2.20.0


In [4]:
True_News = pd.read_csv("True.csv")
True_News['target'] = 1
Fake_News = pd.read_csv("Fake.csv")
Fake_News['target'] = 0

In [5]:
True_News.drop('date', axis =1, inplace =True)

In [6]:
Fake_News.drop('date', axis =1, inplace = True)

In [7]:
df = pd.concat([True_News, Fake_News], ignore_index=True )

In [8]:
df

,title,text,subject,target
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,1
...,...,...,...,...
44893,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,0
44894,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,0
44895,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,0
44896,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,0


In [9]:
df = df.sample(frac = 1.0, ignore_index=True)
df

,title,text,subject,target
0,Two Turkish soldiers killed in northern Iraq: ...,ISTANBUL (Reuters) - Two Turkish soldiers were...,worldnews,1
1,Republican Lawmaker Grabs Woman By The Genita...,Conservatives are now committing sexual assaul...,News,0
2,WATCH: TRUMP PUTS TOGETHER HILARIOUS Montage O...,"So much for the wisdom of Obama, the know-it-a...",politics,0
3,VIRAL VIDEO UPDATE: MARINE VET Has To Be Evacu...,UPDATE VIDEO: Language alert:,politics,0
4,"In bid for history, Clinton fails to reach the...",WASHINGTON (Reuters) - Hillary Clinton came cl...,politicsNews,1
...,...,...,...,...
44893,JUDGE JEANINE ON FIRE: “Paul Ryan Needs to Ste...,,politics,0
44894,OUTRAGE! OBAMA’S FEDERAL Wildlife Officers (?)...,US Border Agents and local law enforcement gav...,left-news,0
44895,VETERAN BIKER GROUP Helps Returning Soldier Fo...,HOW IS IT THAT SQUATTERS HAVE MORE RIGHTS THAN...,Government News,0
44896,Russia U.N. envoy says U.S. sanctions will not...,UNITED NATIONS (Reuters) - New U.S. sanctions ...,politicsNews,1


In [10]:
df.shape

(44898, 4)

In [11]:
X = df.drop('target', axis = 1) 
y = df.target

In [12]:
X.shape ,y.shape

((44898, 3), (44898,))

In [13]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [14]:
voc_size = 5000

In [15]:
news = X.copy()

In [16]:
news['title'][0]

'Two Turkish soldiers killed in northern Iraq: Turkish military'

In [17]:
!pip install nltk


In [18]:
import nltk
import re
from nltk.corpus import stopwords

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Gaurav\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("GPUs available:", tf.config.list_physical_devices('GPU'))


TensorFlow version: 2.20.0
GPUs available: []


In [21]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(news)):
    review = re.sub('[^a-zA-Z]', ' ', news['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [22]:
one_hot_repr = [one_hot(words, voc_size) for words in corpus]
one_hot_repr

[[1242, 3053, 1629, 233, 1158, 4062, 3053, 189],
 [3610, 4032, 928, 3296, 602, 4854, 724, 2070, 1603],
 [2972,
  724,
  3261,
  32,
  4260,
  1788,
  2931,
  725,
  3502,
  2355,
  748,
  567,
  1412,
  1183],
 [878,
  2748,
  1235,
  4204,
  4213,
  4522,
  4062,
  4854,
  2948,
  2769,
  966,
  892,
  2996,
  2748],
 [3824, 4501, 4043, 303, 2817, 601],
 [1018, 3881, 4895, 3774, 2427, 1892, 4580],
 [4087, 4504, 565, 3408, 1446, 894, 78, 1050, 4854, 724],
 [1446, 1732, 946, 2205, 257, 2769, 3064, 1356],
 [724, 931, 2486, 967, 2296, 1807, 2961, 668, 3365],
 [539, 3427, 3274, 567, 285, 271],
 [724, 4292, 2006, 451, 1914, 2931, 890],
 [724, 4008, 4094, 1788, 831, 4347, 4341, 924],
 [724, 3921, 4713, 25, 1892, 2456, 1500, 623, 565, 3738],
 [2070, 2007, 3109, 3272, 2096, 4293, 2306, 761, 1063, 949, 1391, 3893],
 [967, 2409, 553, 1257, 1805, 662, 2163, 724, 4264, 1232],
 [4077, 2006, 1234, 29, 4264, 3926, 1843],
 [724, 3140, 848, 3563, 3159, 761, 2070],
 [4172, 4133, 1373, 1711, 2138, 1732, 

In [23]:
sent_length = 20

In [24]:
embedded_docs = pad_sequences(one_hot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4062 3053  189]
 [   0    0    0 ...  724 2070 1603]
 [   0    0    0 ...  567 1412 1183]
 ...
 [   0    0    0 ... 3727 2477 4470]
 [   0    0    0 ... 2739  779 1875]
 [   0    0    0 ... 1577 4946 3685]]


In [25]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0, 1242, 3053, 1629,  233, 1158, 4062, 3053,  189], dtype=int32)

In [26]:
from tensorflow.keras.layers import Dropout

embedding_vector_features = 40 
model = Sequential()
model.add(Embedding(
    input_dim=voc_size,
    output_dim=embedding_vector_features,
    input_shape=(sent_length,)
))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

C:\Users\Gaurav\Desktop\ML Projects\KrishNaik\env\Lib\site-packages\keras\src\layers\core\embedding.py:100: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 20, 40)              │         200,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 20, 40)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 100)                 │          56,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 256,501 (1001.96 KB)

 Trainable params: 256,501 (1001.96 KB)

 Non-trainable params: 0 (0.00 B)

In [27]:
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [28]:
X_final.shape,y_final.shape

((44898, 20), (44898,))

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [30]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.8956 - loss: 0.2457 - val_accuracy: 0.9374 - val_loss: 0.1539
Epoch 2/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.9499 - loss: 0.1300 - val_accuracy: 0.9410 - val_loss: 0.1529
Epoch 3/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.9614 - loss: 0.1017 - val_accuracy: 0.9443 - val_loss: 0.1450
Epoch 4/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.9678 - loss: 0.0846 - val_accuracy: 0.9451 - val_loss: 0.1549
Epoch 5/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.9735 - loss: 0.0710 - val_accuracy: 0.9438 - val_loss: 0.1702
Epoch 6/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.9776 - loss: 0.0623 - val_accuracy: 0.9394 - val_loss: 0.1818
Epoch 7/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.9799 - loss: 0.0549 - val_accuracy: 0.9440 - val_loss: 0.1874
Epoch 8/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.9825 - loss: 0.0476 - val_accu

In [31]:
y_pred=model.predict(X_test)
y_pred = (y_pred > 0.5)

464/464 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step


In [32]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[7343,  482],
       [ 367, 6625]])

In [33]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9427009516096376